In [23]:
#getting the daaaata
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""Dataset creation and loading."""

from typing import List, Tuple

import numpy as np
import tensorflow as tf
from tensorflow import Tensor
from tensorflow.keras.utils import to_categorical
from termcolor import cprint
from tqdm.auto import tqdm
from glob import glob


# pylint: disable=too-many-positional-arguments
def create_dataset(file_pattern: str,
                   batch_size: int = 32,
                   attack_point: str = "key",
                   attack_byte: int = 0,
                   num_shards: int = 256,
                   num_traces_per_shard: int = 256,
                   max_trace_length: int = 20000,
                   is_training: bool = True,
                   shuffle_size: int = 65535) -> Tuple[Tensor, Tensor]:
    del shuffle_size  # unused
    del is_training  # unused
    del batch_size  # unused

    shards = list_shards(file_pattern, num_shards)
    attack_byte = int(attack_byte)

    if attack_point not in ["key", "sub_bytes_in", "sub_bytes_out"]:
        raise ValueError(
            "invalid attack point. avail: key, sub_bytes_in, sub_bytes_out")

    x_list: List[Tensor] = []
    y_list: List[Tensor] = []
    pb = tqdm(total=num_shards, desc="loading shards")
    with tf.device("/cpu:0"):
        for idx, shard_fname in enumerate(shards):
            x_shard, y_shard = load_shard(shard_fname, attack_byte,
                                          attack_point, max_trace_length,
                                          num_traces_per_shard)

            del idx  # unused
            # if not idx:
            #     x = x_shard
            #     y = y_shard
            # else:
            #     x = tf.concat([x, x_shard], axis=0)
            #     y = tf.concat([y, y_shard], axis=0)
            x_list.append(x_shard)
            y_list.append(y_shard)
            pb.update()
        pb.close()
        x: Tensor = tf.concat(x_list, axis=0)
        y: Tensor = tf.concat(y_list, axis=0)

    cprint("[Generator]", "yellow")
    cprint(f"|-attack point:{attack_point}", "blue")
    cprint(f"|-attack byte:{attack_byte}", "green")
    cprint(f"|-num shards:{num_shards}", "blue")
    cprint(f"|-traces per shards:{num_traces_per_shard}", "green")
    cprint(f"|-y:{str(y.shape)}", "blue")
    cprint(f"|-x:{str(x.shape)}", "green")

    # make it a tf dataset
    # cprint("building tf dataset", "magenta")
    # dataset = tf.data.Dataset.from_tensor_slices((x, y))
    # dataset.cache()
    # if is_training:
    #     dataset = dataset.shuffle(shuffle_size, reshuffle_each_iteration=True)
    # dataset = dataset.batch(batch_size).prefetch(
    #     tf.data.experimental.AUTOTUNE
    # )
    return (x, y)


def list_shards(file_pattern: str, num_shards: int) -> List[str]:
    return glob(file_pattern)[:num_shards]


# pylint: disable=too-many-positional-arguments
def load_attack_shard(
        fname: str,
        attack_byte: int,
        attack_point: str,
        max_trace_length: int,
        num_traces: int = 256,
        full_key: bool = False) -> Tuple[bytearray, bytearray, Tensor, Tensor]:
    """Load a shard of data that target a given key

    Args:
        fname ([type]): [description]
        attack_byte ([type]): [description]
        attack_point ([type]): [description]
        max_trace_length ([type]): [description]
        num_traces (int, optional): [description]. Defaults to 256.

    Returns:
        list: keys, pts, attack_points_val, power_traces
    """
    del full_key  # unused
    shard = np.load(fname)
    attack_byte = int(attack_byte)

    # key
    k = shard["keys"][attack_byte][:num_traces]
    pts = shard["pts"][attack_byte][:num_traces]
    # load y
    if attack_point == "key":
        y = shard["keys"][attack_byte]
    elif attack_point == "sub_bytes_in":
        y = shard["sub_bytes_in"][attack_byte]
    elif attack_point == "sub_bytes_out":
        y = shard["sub_bytes_out"][attack_byte]
    else:
        raise ValueError(f"Unknown attack point {attack_point}.")

    y = y[:num_traces]
    y = to_categorical(y, 256)
    y = tf.convert_to_tensor(y, dtype="uint8")

    # load x
    x = shard["traces"][:num_traces, :max_trace_length, :]
    x = tf.convert_to_tensor(x, dtype="float32")
    return k, pts, x, y


def load_shard(fname: str, attack_byte: int, attack_point: str,
               max_trace_length: int,
               num_traces_per_shard: int) -> Tuple[Tensor, Tensor]:
    shard = np.load(fname)

    # load y
    if attack_point == "key":
        y = shard["keys"][attack_byte]
    elif attack_point == "sub_bytes_in":
        y = shard["sub_bytes_in"][attack_byte]
    elif attack_point == "sub_bytes_out":
        y = shard["sub_bytes_out"][attack_byte]
    else:
        raise ValueError(f"Unknown attack point {attack_point}.")

    y = y[:num_traces_per_shard]
    y = to_categorical(y, 256)
    y = tf.convert_to_tensor(y, dtype="uint8")

    # load x
    x = shard["traces"][:num_traces_per_shard, :max_trace_length, :]
    x = tf.convert_to_tensor(x, dtype="float32")
    return x, y

In [30]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""Intro model."""

from typing import Any, Dict, Tuple

from tensorflow import Tensor
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam

from scaaml.utils import display_config
from scaaml.utils import get_num_gpu


# pylint: disable=too-many-positional-arguments
def block(x: Tensor,
          filters: int,
          kernel_size: int = 3,
          strides: int = 1,
          conv_shortcut: bool = False,
          activation: str = "relu") -> Tensor:
    """Residual block with pre-activation
    From: https://arxiv.org/pdf/1603.05027.pdf

    Args:
        x: input tensor.
        filters (int): filters of the bottleneck layer.

        kernel_size(int, optional): kernel size of the bottleneck layer.
        defaults to 3.

        strides (int, optional): stride of the first layer.
        defaults to 1.

        conv_shortcut (bool, optional): Use convolution shortcut if True,
        otherwise identity shortcut. Defaults to False.

        use_batchnorm (bool, optional): Use batchnormalization if True.
        Defaults to True.

        activation (str, optional): activation function. Defaults to "relu".

    Returns:
        Output tensor for the residual block.
    """

    x = layers.BatchNormalization()(x)
    x = layers.Activation(activation)(x)

    if conv_shortcut:
        shortcut = layers.Conv1D(4 * filters, 1, strides=strides)(x)
    else:
        if strides > 1:
            shortcut = layers.MaxPooling1D(1, strides=strides)(x)
        else:
            shortcut = x

    x = layers.Conv1D(filters, 1, use_bias=False, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(activation)(x)

    x = layers.Conv1D(filters,
                      kernel_size,
                      strides=strides,
                      use_bias=False,
                      padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(activation)(x)

    x = layers.Conv1D(4 * filters, 1)(x)
    x = layers.Add()([shortcut, x])
    return x


# pylint: disable=too-many-positional-arguments
def stack(x: Tensor,
          filters: int,
          blocks: int,
          kernel_size: int = 3,
          strides: int = 2,
          activation: str = "relu") -> Tensor:
    """A set of stacked residual blocks.
    Args:
        filters (int): filters of the bottleneck layer.

        blocks (int): number of conv blocks to stack.

        kernel_size(int, optional): kernel size of the bottleneck layer.
        defaults to 3.

        strides (int, optional): stride used in the last block.
        defaults to 2.

        conv_shortcut (bool, optional): Use convolution shortcut if True,
        otherwise identity shortcut. Defaults to False.

        activation (str, optional): activation function. Defaults to "relu".

    Returns:
        tensor:Output tensor for the stacked blocks.
  """
    x = block(x,
              filters,
              kernel_size=kernel_size,
              activation=activation,
              conv_shortcut=True)
    for _ in range(2, blocks):
        x = block(x, filters, kernel_size=kernel_size, activation=activation)
    x = block(x, filters, strides=strides, activation=activation)
    return x


from typing import Any, Dict, Tuple
from tensorflow.keras.models import Model


# pylint: disable=C0103
def Resnet1D(input_shape: Tuple[int, ...], attack_point: str,
             mdl_cfg: Dict[str, Any], optim_cfg: Dict[str,
                                                      Any]) -> Model:
    del attack_point  # unused

    pool_size = mdl_cfg["initial_pool_size"]
    filters = mdl_cfg["initial_filters"]
    block_kernel_size = mdl_cfg["block_kernel_size"]
    activation = mdl_cfg["activation"]
    dense_dropout = mdl_cfg["dense_dropout"]
    num_blocks = [
        mdl_cfg["blocks_stack1"], mdl_cfg["blocks_stack2"],
        mdl_cfg["blocks_stack3"], mdl_cfg["blocks_stack4"]
    ]

    inputs = layers.Input(shape=input_shape)
    x = inputs

    # stem
    x = layers.MaxPool1D(pool_size=pool_size)(x)

    # trunk: stack of residual block
    for block_idx in range(4):
        filters *= 2
        x = stack(x,
                  filters,
                  num_blocks[block_idx],
                  kernel_size=block_kernel_size,
                  activation=activation)

    # head model: dense
    x = layers.GlobalAveragePooling1D()(x)
    for _ in range(1):
        x = layers.Dropout(dense_dropout)(x)
        x = layers.Dense(256)(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation(activation)(x)

    outputs = layers.Dense(256, activation="softmax")(x)

    model: Model[Any, Any] = Model(inputs=inputs, outputs=outputs)
    model.summary()

    if get_num_gpu() > 1:
        lr = optim_cfg["multi_gpu_lr"]
    else:
        lr = optim_cfg["lr"]

    model.compile(loss=["categorical_crossentropy"],
                  metrics=["acc"],
                  optimizer=Adam(lr))
    return model


def get_model(input_shape: Tuple[int, ...], attack_point: str,
              config: Dict[str, Any]) -> Model:
    """Return an instantiated model based of the config provided.

    Args:
        config (dict): scald config.
    """

    mdl_cfg = config["model_parameters"]
    optim_cfg = config["optimizer_parameters"]

    display_config("model", mdl_cfg)
    display_config("optimizer", optim_cfg)
    return Resnet1D(input_shape, attack_point, mdl_cfg, optim_cfg)

In [31]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""Model training."""

import argparse
import json
import sys
import zipfile
import os
from termcolor import cprint

import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
import tensorflow.keras.backend as K

from scaaml.intro.generator import create_dataset
#from scaaml.intro.model import get_model
from scaaml.utils import get_model_stub
from scaaml.utils import get_num_gpu
from scaaml.utils import tf_cap_memory


def train_model(config):
    algorithm = config["algorithm"]
    train_glob = f"datasets/{algorithm}/train/*"
    test_glob = f"datasets/{algorithm}/test/*"
    test_shards = 256
    num_traces_per_test_shards = 16
    batch_size = config["batch_size"] * get_num_gpu()

    for attack_byte in config["attack_bytes"]:
        for attack_point in config["attack_points"]:

            x_train, y_train = create_dataset(
                train_glob,
                batch_size=batch_size,
                attack_point=attack_point,
                attack_byte=attack_byte,
                num_shards=config["num_shards"],
                num_traces_per_shard=config["num_traces_per_shard"],
                max_trace_length=config["max_trace_len"],
                is_training=True)

            x_test, y_test = create_dataset(
                test_glob,
                batch_size=batch_size,
                attack_point=attack_point,
                attack_byte=attack_byte,
                num_shards=test_shards,
                num_traces_per_shard=num_traces_per_test_shards,
                max_trace_length=config["max_trace_len"],
                is_training=False)

            # infers shape
            input_shape = x_train.shape[1:]

            # reset graph and load a new model
            K.clear_session()

            # display config
            cprint(f"[{algorithm}]", "magenta")
            cprint(">Attack params", "green")
            cprint(f"|-attack_point:{attack_point}", "cyan")
            cprint(f"|-attack_byte:{attack_byte}", "yellow")
            cprint(f"|-input_shape:{str(input_shape)}", "cyan")

            # multi gpu
            #strategy = tf.distribute.MirroredStrategy()
            #with strategy.scope():
                #model = get_model(input_shape, attack_point, config)
            mdl_cfg = config["model_parameters"]
            optim_cfg = config["optimizer_parameters"]
            display_config("model", mdl_cfg)
            display_config("optimizer", optim_cfg)
            model = Resnet1D(input_shape, attack_point, mdl_cfg, optim_cfg)
                # model recording setup
            stub = get_model_stub(attack_point, attack_byte, config)
            cb = [
                ModelCheckpoint(monitor="val_loss",
                                filepath=f"models/{stub}.keras",
                                save_best_only=True),
                                TensorBoard(log_dir="logs/" + stub, update_freq="batch")
            ]

            model.fit(x_train,
                      y_train,
                      validation_data=(x_test, y_test),
                      verbose=1,
                      epochs=config["epochs"],
                      callbacks=cb)
config = """{
    "model": "cnn",
    "device": "stm32f0",
    "algorithm": "tinyaes",
    "version": "10",
    "attack_points": [
        "sub_bytes_in"

    ],
    "attack_bytes": [
        "0"
    ],
    "max_trace_len": 5000,
    "num_shards": 256,
    "num_traces_per_shard": 256,
    "batch_size": 32,
    "epochs": 30,
    "optimizer_parameters": {
        "lr": 0.001,
        "multi_gpu_lr": 0.001
    },
    "model_parameters": {
        "activation": "relu",
        "initial_filters": 8,
        "initial_pool_size": 4,
        "block_kernel_size": 3,
        "blocks_stack1": 3,
        "blocks_stack2": 4,
        "blocks_stack3": 4,
        "blocks_stack4": 3,
        "dense_dropout": 0.1
    }
}"""

if __name__ == "__main__":
    #parser = argparse.ArgumentParser(description="Train models")
   # parser.add_argument("--config", "-c", default="config", help="Train config")
    #args = parser.parse_args()
   # if not args.config:
   #     parser.print_help()
   #     sys.exit()
   # with open(args.config, encoding="utf-8") as config_file:
   #     train_model(json.loads(config_file.read()))
   train_model(json.loads(config))

loading shards: 100%|██████████| 256/256 [00:18<00:00, 13.71it/s]


[Generator]
|-attack point:sub_bytes_in
|-attack byte:0
|-num shards:256
|-traces per shards:256
|-y:(65536, 256)
|-x:(65536, 5000, 1)


loading shards: 100%|██████████| 256/256 [00:18<00:00, 13.51it/s]


[Generator]
|-attack point:sub_bytes_in
|-attack byte:0
|-num shards:256
|-traces per shards:16
|-y:(4096, 256)
|-x:(4096, 5000, 1)
[tinyaes]
>Attack params
|-attack_point:sub_bytes_in
|-attack_byte:0
|-input_shape:(5000, 1)
[model]
activation:relu
initial_filters:8
initial_pool_size:4
block_kernel_size:3
blocks_stack1:3
blocks_stack2:4
blocks_stack3:4
blocks_stack4:3
dense_dropout:0.1
[optimizer]
lr:0.001
multi_gpu_lr:0.001


Model: "functional"

...

 Total params: 1,122,388 (4.28 MB)

 Trainable params: 1,113,106 (4.25 MB)

 Non-trainable params: 9,282 (36.26 KB)

Epoch 1/30
 366/2048 ━━━━━━━━━━━━━━━━━━━━ 11:46 420ms/step - acc: 0.0037 - loss: 5.7200

KeyboardInterrupt: 